In [132]:
#!pip install open3d
#!pip install rospkg

In [133]:
#!unzip aruco_unedited.zip

In [134]:
import numpy as np
from PIL import Image
from markerDetector import detect_markers, calculate_camera_pose
import cv2 as cv
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import cv2.aruco as aruco
import os
import re

In [136]:
def threshold(image, low, high, channel):
  filtered_image = np.array(image, copy=True)
  filtered_image[filtered_image[:, :, channel] < low] = 0
  filtered_image[filtered_image[:, :, channel] > high] = 0
  return filtered_image

In [137]:
def cut_threshold(image, low, high, channel):
  filtered_image = np.array(image, copy=True)
  cut = np.array(image, copy=True)
  filtered_image[filtered_image[:, :, channel] < low] = 0
  filtered_image[filtered_image[:, :, channel] > high] = 0
  cut = cut - filtered_image
  return cut

In [138]:
def cut_threshold_difference(image, channel_main, channel_2, channel_3, alpha2, alpha3, delta):
  filtered_image = np.array(image, copy=True, dtype='int32')
  filtered_image[:, :, channel_main] = filtered_image[:, :, channel_main] - alpha2 * filtered_image[:, :, channel_2] - alpha3 * filtered_image[:, :, channel_3] - delta
  filtered_image[filtered_image[:, :, channel_main] < 0] = 0
  filtered_image[:, :, channel_2] = 0
  filtered_image[:, :, channel_3] = 0
  filtered_image[filtered_image[:, :, channel_main] > 0] = 255
  return np.array(filtered_image, dtype='uint8')

In [139]:
def make_binary(image):
  filtered_image = np.array(image, copy=True)
  filtered_image[filtered_image[:, :, 0] > 0] = 255
  filtered_image[filtered_image[:, :, 1] > 0] = 255
  filtered_image[filtered_image[:, :, 2] > 0] = 255
  return filtered_image

In [140]:
def closest(corners, point):
  arr1 = []
  arr0 = []
  min = 0
  for i in range(len(corners)):
    arr1.append(int(corners[i][0][0][1]))
    arr0.append(int(corners[i][0][0][0]))
  min_distance = np.sqrt((arr1[0] - point[1]) ** 2 + (arr0[0] - point[0]) ** 2)
  for i in range(1, len(corners)):
    if min_distance > np.sqrt((arr1[i] - point[1]) ** 2 + (arr0[i] - point[0]) ** 2):
      min_distance = np.sqrt((arr1[i] - point[1]) ** 2 + (arr0[i] - point[0]) ** 2)
      min = i
  return min, min_distance

In [141]:
def get2D(T, K):
  T = T.reshape((4, 4))
  T = np.linalg.inv(T)
  T=T[0:3,:]
  K = K.reshape((3, 3))
  P=K@T
  point3D=np.array([55,25,0,1])
  point2D=P@point3D
  point2D[0] /= point2D[2]
  point2D[1] /= point2D[2]
  point2D[2] /= point2D[2]
  return [int(point2D[0]), int(point2D[1])]

In [142]:
def segment_circle(image, corners, point):
  filtered_image = np.array(image, copy=True)
  _, radius = closest(corners, point)
  cv.circle(filtered_image, point, int(radius), (255, 255, 255), -1)
  filtered_image[filtered_image[:, :, 0] < 255] = 0
  filtered_image[filtered_image[:, :, 1] < 255] = 0
  filtered_image[filtered_image[:, :, 2] < 255] = 0
  cropped = np.array(image, copy=True)
  cropped = cv.bitwise_and(cropped, filtered_image)
  return cropped

In [143]:
def get_marked_image(image, corners):
  filtered_image = np.array(image, copy=True)
  filtered_image[:, :, :] = 0
  for i in range(len(corners)):
    for j in range(1):
      filtered_image[int(corners[i][0][j][1]), int(corners[i][0][j][0]), 0] = 255
      filtered_image[int(corners[i][0][j][1]), int(corners[i][0][j][0]), 1] = 255
      filtered_image[int(corners[i][0][j][1]), int(corners[i][0][j][0]), 2] = 255
      #min = closest(corners, i)
      for k in range(len(corners)):
        if i != k:
          cv.line(filtered_image,(int(corners[i][0][j][0]),int(corners[i][0][j][1])),(int(corners[k][0][j][0]),int(corners[k][0][j][1])),(255,255,255),3)
        else:
          i+=1

  for i in range(len(filtered_image)):
    mark1 = False
    mark2 = False
    index_last = 0
    for j in range(len(filtered_image[0])):
      if filtered_image[i][j][0] == 255 and not mark2:
        mark1 = True
      if filtered_image[i][j][0] == 255 and mark2:
        mark2 = False
        mark1 = False
      if filtered_image[i][j][0] == 0 and mark1 and not mark2:
        mark2 = True
        index_last = j
      if mark1 and mark2:
        filtered_image[i][j][:] = 255
    for j in range(len(filtered_image[0]) - 1, index_last, -1):
      filtered_image[i][j][:] = 0

  cropped = np.array(image, copy=True)
  cropped = cv.bitwise_and(cropped, filtered_image)
  return cropped

In [ ]:
K = np.array([920.88464355, 0, 624.062744140625,
    0, 924.34155273, 374.5955547,
    0, 0, 1])

poses = []
with open('poses.txt') as f:
  contents = f.readlines()
  for content in contents:
    poses.append(re.findall(r"[-+]?\d*\.\d+|\d+", content))
poses = np.array(poses, dtype=float)

for i in range(1, 9):
  img_name = "aruco_unedited/" + "image_" + str(i) + ".jpeg"
  img = cv.imread(img_name)
  assert img is not None, "file could not be read, check with os.path.exists()"
  image, markerCorners, markerIds = detect_markers(np.array(img, copy=True))
  img = segment_circle(img, markerCorners, get2D(poses[i - 1], K))
  img = cv.blur(img,(5,5))
  # Reshaping the image into a 2D array of pixels and 3 color values (RGB)
  pixel_vals = img.reshape((-1,3))

  # Convert to float type
  pixel_vals = np.float32(pixel_vals)

  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 500, 0.85)

  # then perform k-means clustering with number of clusters defined as 3
  #also random centres are initially choosed for k-means clustering
  k = 15
  retval, labels, centers = cv.kmeans(pixel_vals, k, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)

  # convert data into 8-bit values
  centers = np.uint8(centers)
  segmented_data = centers[labels.flatten()]

  # reshape data into the original image dimensions
  segmented_image = segmented_data.reshape((img.shape))
  filtered_image = np.array(segmented_image, copy=True)
  filtered_image = cut_threshold_difference(filtered_image, 2, 1, 0, 1, 0, 50)
  binary = make_binary(filtered_image)

  im = Image.fromarray(binary).convert('L')
  im.save("segmented_images/" + 'image' + str(i) + '.png')
  print("image" + str(i) + " done!")


image1 done!
image2 done!
image3 done!
image4 done!
image5 done!
image6 done!
image7 done!


In [ ]:
import shutil
shutil.make_archive("segmented_images", 'zip', "segmented_images")